In [0]:
%sql
-- Load raw PDF files from the specified directory and create a temporary view
CREATE OR REPLACE TEMP VIEW raw_unstructured_files_data AS
SELECT 
  path,
  content
FROM read_files('/Volumes/idp/default/final_project');

In [0]:
%sql
-- Parse document content using AI and create a table with parsed results
CREATE OR REPLACE TABLE parsed_content_data AS
SELECT 
   *
  ,ai_parse_document(content) as parsed_content
FROM raw_unstructured_files_data;

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Extract and concatenate document elements into a single string for each file
CREATE OR REPLACE TABLE pretty_html_data AS
SELECT
  path,
  concat_ws('/n',
    transform(try_cast(parsed_content:document:elements AS ARRAY<VARIANT>), e -> coalesce(try_cast(e:content as string), ''))
  ) as document
FROM parsed_content_data;

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Classify each document using AI based on its content
create or replace table classified_html_data as
select 
   *
  ,ai_classify(document, array('Invoice', 'Purchase Order', 'accessories', 'Receipt', 'Others')) as doc_classification
from pretty_html_data

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Extract invoice fields from classified documents and create a table for invoices
CREATE OR REPLACE TABLE tb_invoice_data
select
  *
  ,ai_extract(document, 
            array('Vendor_Name','Invoice_Number', 'Invoice_Date', 'Due_Date', 'Payment_Method','Total')
            ) as invoice_extracted
 from classified_html_data
where doc_classification = 'Invoice';

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Create schema for finance tables if it does not exist
CREATE SCHEMA if not exists idp.finance;

In [0]:
%sql
-- Summarize invoice data and store in finance schema
create or replace table idp.finance.tb_invoice_data_summary as
select 
  path,
  invoice_extracted.Vendor_Name as vendor_name,
  invoice_extracted.Invoice_Number as invoice_number,
  invoice_extracted.Invoice_Date as invoice_date,
  invoice_extracted.Due_Date as due_date,
  invoice_extracted.Payment_Method as payment_method,
  invoice_extracted.Total as total
from tb_invoice_data

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Extract purchase order fields from classified documents and create a table for purchase orders
CREATE OR REPLACE TABLE tb_purchase_order_data
select
  *
  ,ai_extract(document, 
            array('Merchant_Name','Purcahse_Order_Number', 'Purcahse_Order_Date', 'Requested_Ship_Date',
            'Buyer_Address', 'Vendor_Address', 'Currency', 'Subtotal', 'Shipping_Charge', 'Total')
            ) as po_extracted
 from classified_html_data
where doc_classification = 'Purchase Order';

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Summarize purchase order data and store in finance schema
create or replace table idp.finance.tb_purchase_order_data_summary as
select 
  path,
  po_extracted.Merchant_Name as merchant_name,
  po_extracted.Purcahse_Order_Number as po_number,
  po_extracted.Purcahse_Order_Date as po_date,
  po_extracted.Requested_Ship_Date as ship_date,
  po_extracted.Buyer_Address as buyer_address,
  po_extracted.Vendor_Address as vendor_address,
  po_extracted.Currency as currency,
  po_extracted.Subtotal as subtotal,
  po_extracted.Shipping_Charge as shipping_charge,
  po_extracted.Total as total  
from tb_purchase_order_data

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Extract receipt fields from classified documents and create a table for receipts
CREATE OR REPLACE TABLE tb_receipt_data
select
  *
  ,ai_extract(document, 
            array('Merchant_Name','Receipt_Number', 'Receipt_Date', 'Receipt_Time', 'Receipt_Address', 'Payment_Method','Return_Policy', 'Total')
            ) as receipt_extracted
 from classified_html_data
where doc_classification = 'Receipt';

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Summarize receipt data and store in finance schema
create or replace table idp.finance.tb_receipt_data_summary as
select 
  path,
  receipt_extracted.Merchant_Name as merchant_name,
  receipt_extracted.Receipt_Number as receipt_number,
  receipt_extracted.Receipt_Date as receipt_date,
  receipt_extracted.Receipt_Time as receipt_time,
  receipt_extracted.Receipt_Address as receipt_address,
  receipt_extracted.Total as receipt_total
from tb_receipt_data;

num_affected_rows,num_inserted_rows
